# PG AI - Natural Language Processing and Speech Recognition
# Project: Build a speech-to-text Model with TensorFlow Dataset

#### DESCRIPTION

Using NLP and machine learning, make a model to predict the rating in a review based on the content of the text review. This will help identify cases with a mismatch.<br>

#### Problem Statement:  

Zomato is India’s largest platform for discovering restaurants and ordering food. It operates in India as well as a few cities internationally. Bangalore is one of the biggest customers and restaurant bases for Zomato with 4 to 5 million users using the platform each month.<br>

Users on the platform can also post reviews of restaurants and provide a rating accompanying the review. The content in the reviews should ideally reflect the rating provided by the customer. In many cases, there is a mismatch, owing to multiple reasons, where the rating does not match the customer review. The reviews and rating match is very important as it builds customer trust on the platform and helps the user get an accurate picture of the restaurant.<br> 

You, as a data scientist, need to enable the identification and cleanup of such cases to ensure the ratings reflect the reviews and that the reviews seem trustworthy to the customer. You will need to use NLP techniques in conjunction with machine learning models to predict the rating from the review text. <br>

#### Domain: 
Hospitality and internet

Analysis to be done: Perform specific data cleanup, build a rating prediction model using the Random Forest technique and NLP. 

#### Content: 

rating:
the rating given by the customer

review_text: the text in the review

#### Steps to perform:

Perform clean up on the data; tweak the stop words (negative terms are important). Follow up with a Random Forest Regressor to predict the star rating given by the customers.

#### Tasks: 

1. Load the data using read_csv function from pandas package
2. Null values in the review text? 
    1. Remove the records where the review text is null
3. Perform cleanup on the data 
    1. Normalize the casing
    2. Remove extra line breaks from the text
    3. Remoove stop words
        1. Note: Terms like ‘no’, ‘not’, ‘don’, ‘won’ are important, don’t remove
    4. Remove punctuation
4. Separation into train and test sets
    1. Use train-test method to divide your data into 2 sets: train and test
    2. Use a 70-30 split
5. Use TF-IDF values for the terms as features to get into a vector space model
    1. Import TF-IDF vectorizer from sklearn
    2. Instantiate with a maximum of 5000 terms in your vocabulary
    3. Fit and apply on the train set
    4. Apply on the test set
6. Model building: Random Forest Regressor
    1. Instantiate RandomForestRegressor from sklearn (set random seed)
    2. Fit on the train data
    3. Make predictions for the train set
7. Model evaluation
    1. Report the root mean square error
8. Hyperparameter tuning
    1. Import GridSearch
    2. Provide the parameter grid to choose:
        1. max_features – ‘auto’, ‘sqrt’, ‘log2’
        2. max_depth – 10, 15, 20, 25
9. Find the parameters with the best mean square error in cross-validation
    1. Choose the appropriate scoring as the metric for scoring
    2. Choose stratified 5 fold cross-validation scheme
    3. Fit on the train set
10. What are the best parameters?
11. Predict and evaluate using the best estimator
    1. Use the best estimator from the grid search to make predictions on the test set
    2. What is the root mean squared error on the test set?
12. Can you identify mismatch cases? 
    1. Make a rule based on the predicted value and the actual value that identifies mismatch cases (e.g. difference in actual and predicted being more than a cutoff)
    2. How many such cases do you see?
    3. Are all these mismatch cases genuine?

By Edson Teixeira<br>
teixeiraedson252@gmail.com <br>
December 29th 2021

In [1]:
import pandas as pd, numpy as np
import re

In [2]:
reviews0 = pd.read_csv("Zomato_reviews.csv")

In [3]:
reviews0.head()

,rating,review_text
0,1.0,"Their service is worst, pricing in menu is dif..."
1,5.0,really appreciate their quality and timing . I...
2,4.0,"Went there on a Friday night, the place was su..."
3,4.0,A very decent place serving good food.\r\nOrde...
4,5.0,One of the BEST places for steaks in the city....


In [4]:
reviews0.describe(include="all")

,rating,review_text
count,27762.000000,27748
unique,NaN,10548
top,NaN,good
freq,NaN,278
mean,3.665784,NaN
std,1.284573,NaN
min,1.000000,NaN
25%,3.000000,NaN
50%,4.000000,NaN
75%,5.000000,NaN


14 rows are missing the review text - need to get rid of these records

In [5]:
reviews1 = reviews0[~reviews0.review_text.isnull()].copy()
reviews1.reset_index(inplace=True, drop=True)

In [6]:
reviews0.shape, reviews1.shape

((27762, 2), (27748, 2))

#### Converting to list for easy manipulation

In [7]:
reviews_list = reviews1.review_text.values

In [8]:
len(reviews_list)

27748

### Text clean up 
- Normalize the case  
- Remove stop words
   - remove "not", "no" from the stop word list
- Remove punctuations

Normalizing case

In [9]:
reviews_lower = [txt.lower() for txt in reviews_list]

In [10]:
reviews_lower[2:4]

['went there on a friday night, the place was surprisingly empty. interesting menu which is almost fully made of dosas. i had bullseye dosa and cheese masala dosa. the bullseye dosa was really good, with the egg perfectly cooked to a half boiled state. the masala in the cheese masala was good, but the cheese was a bit too chewy for my liking. the chutney was good, the sambar was average. the dishes are reasonably priced.',
 'a very decent place serving good food.\r\nordered chilli fish, chicken & pork sizzler.\r\neverything tasted good but pork could have been slightly better cooked.\r\ntried 2 beverages, both were very sweet.']

Remove extra line breaks

In [11]:
reviews_lower = [" ".join(txt.split()) for txt in reviews_lower]

In [12]:
reviews_lower[2:4]

['went there on a friday night, the place was surprisingly empty. interesting menu which is almost fully made of dosas. i had bullseye dosa and cheese masala dosa. the bullseye dosa was really good, with the egg perfectly cooked to a half boiled state. the masala in the cheese masala was good, but the cheese was a bit too chewy for my liking. the chutney was good, the sambar was average. the dishes are reasonably priced.',
 'a very decent place serving good food. ordered chilli fish, chicken & pork sizzler. everything tasted good but pork could have been slightly better cooked. tried 2 beverages, both were very sweet.']

#### Tokenize

In [13]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Fortnite\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [14]:
print(word_tokenize(reviews_lower[0]))

['their', 'service', 'is', 'worst', ',', 'pricing', 'in', 'menu', 'is', 'different', 'from', 'bill', '.', 'they', 'can', 'give', 'you', 'a', 'bill', 'with', 'increased', 'pricing', '.', 'even', 'for', 'serving', 'water', ',', 'menu', ',', 'order', 'you', 'need', 'to', 'call', 'them', '3-4', 'times', 'even', 'on', 'a', 'non', 'busy', 'day', '.']


In [15]:
reviews_tokens = [word_tokenize(sent) for sent in reviews_lower]
print(reviews_tokens[0])

['their', 'service', 'is', 'worst', ',', 'pricing', 'in', 'menu', 'is', 'different', 'from', 'bill', '.', 'they', 'can', 'give', 'you', 'a', 'bill', 'with', 'increased', 'pricing', '.', 'even', 'for', 'serving', 'water', ',', 'menu', ',', 'order', 'you', 'need', 'to', 'call', 'them', '3-4', 'times', 'even', 'on', 'a', 'non', 'busy', 'day', '.']


### Remove stop words and punctuations

In [16]:
from nltk.corpus import stopwords
from string import punctuation

In [17]:
stop_nltk = stopwords.words("english")
stop_punct = list(punctuation)

In [18]:
print(stop_nltk)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [19]:
stop_nltk.remove("no")
stop_nltk.remove("not")
stop_nltk.remove("don")
stop_nltk.remove("won")

In [20]:
"no" in stop_nltk

False

In [21]:
stop_final = stop_nltk + stop_punct + ["...", "``","''", "====", "must"]

In [22]:
def del_stop(sent):
    return [term for term in sent if term not in stop_final]

In [23]:
del_stop(reviews_tokens[1])

['really',
 'appreciate',
 'quality',
 'timing',
 'tried',
 'thattil',
 'kutti',
 'dosa',
 "'ve",
 'addicted',
 'dosa',
 'really',
 'chutney',
 'really',
 'good',
 'money',
 'worth',
 'much',
 'better',
 'thattukada',
 'try']

In [24]:
reviews_clean = [del_stop(sent) for sent in reviews_tokens]

In [25]:
reviews_clean = [" ".join(sent) for sent in reviews_clean]
reviews_clean[:2]

['service worst pricing menu different bill give bill increased pricing even serving water menu order need call 3-4 times even non busy day',
 "really appreciate quality timing tried thattil kutti dosa 've addicted dosa really chutney really good money worth much better thattukada try"]

### Separate X and Y and perform train test split, 70-30

In [26]:
len(reviews_clean)

27748

In [27]:
X = reviews_clean
y = reviews1.rating

Train test split

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state=42)

### Document term matrix using TfIdf

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [30]:
vectorizer = TfidfVectorizer(max_features = 5000)

In [31]:
len(X_train), len(X_test)

(19423, 8325)

In [32]:
X_train_bow = vectorizer.fit_transform(X_train)

In [33]:
X_test_bow = vectorizer.transform(X_test)

In [34]:
X_train_bow.shape, X_test_bow.shape

((19423, 5000), (8325, 5000))

### Model building

In [35]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

In [36]:
?RandomForestRegressor

In [37]:
learner_rf = RandomForestRegressor(random_state=42)

In [38]:
%%time
learner_rf.fit(X_train_bow, y_train)

Wall time: 6min 43s


RandomForestRegressor(random_state=42)

In [39]:
y_train_preds = learner_rf.predict(X_train_bow)

In [40]:
from sklearn.metrics import mean_squared_error

In [41]:
mean_squared_error(y_train, y_train_preds)**0.5

0.23720347586757728

#### Increase the number of trees

In [42]:
learner_rf = RandomForestRegressor(random_state=42, n_estimators=20)

In [43]:
%%time
learner_rf.fit(X_train_bow, y_train)

Wall time: 1min 19s


RandomForestRegressor(n_estimators=20, random_state=42)

In [44]:
y_train_preds = learner_rf.predict(X_train_bow)

In [45]:
mean_squared_error(y_train, y_train_preds)**0.5

0.2507584827360229

### Hyper-parameter tuning

"class_weights" was one of the many hyperparameters to tune for the SVM.  

Let's find the best hyper-parameters for the SVM classifier

In [46]:
from sklearn.model_selection import GridSearchCV

In [47]:
?RandomForestRegressor

Instantiate the learner with a random state

In [48]:
learner_rf = RandomForestRegressor(random_state=42)

In [49]:
# Create the parameter grid based on the results of random search 
param_grid = {
    'max_features': [500, "sqrt", "log2", "auto"],
    'max_depth': [10, 15, 20, 25]
}

In [50]:
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = learner_rf, param_grid = param_grid, 
                          cv = 5, n_jobs = -1, verbose = 1, scoring = "neg_mean_squared_error" )

In [51]:
grid_search.fit(X_train_bow, y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [10, 15, 20, 25],
                         'max_features': [500, 'sqrt', 'log2', 'auto']},
             scoring='neg_mean_squared_error', verbose=1)

In [53]:
grid_search.cv_results_

{'mean_fit_time': array([  6.74867921,   1.22219944,   0.58779631,  58.66512151,
         15.04919705,   2.26479945,   0.85859833, 102.586408  ,
         23.29021144,   3.41018662,   1.13018107, 147.58514905,
         32.17802038,   4.70092025,   1.47145724, 172.76807294]),
 'std_fit_time': array([1.20657767e-01, 2.07682764e-02, 2.56013859e-03, 9.97178069e-01,
        1.39555736e-01, 1.81466166e-01, 1.15846183e-02, 1.79787131e+00,
        4.76373187e-01, 1.34993868e-01, 1.04844939e-02, 3.11189525e+00,
        5.27870770e-01, 2.19607306e-01, 4.62289179e-02, 2.88618824e+01]),
 'mean_score_time': array([0.10120068, 0.08299899, 0.07999873, 0.09360189, 0.10840015,
        0.11500087, 0.09919982, 0.10613956, 0.1351759 , 0.10937715,
        0.10312624, 0.11807513, 0.14415092, 0.13750024, 0.11562572,
        0.1093698 ]),
 'std_score_time': array([1.36445616e-02, 1.26553768e-03, 4.33455980e-03, 7.96430287e-03,
        3.87934817e-03, 1.96774720e-02, 8.23036380e-03, 6.49581865e-03,
        1.84

In [54]:
grid_search.best_estimator_

RandomForestRegressor(max_depth=25, max_features=500, random_state=42)

### Using the best estimator to make predictions on the test set

In [55]:
y_train_pred = grid_search.best_estimator_.predict(X_train_bow)

In [56]:
y_test_pred = grid_search.best_estimator_.predict(X_test_bow)

In [57]:
mean_squared_error(y_train, y_train_pred)**0.5

0.5876681060137491

In [58]:
mean_squared_error(y_test, y_test_pred)**0.5

0.6715338514796404

### Identifying mismatch cases

In [59]:
res_df = pd.DataFrame({'review':X_test, 'rating':y_test, 'rating_pred':y_test_pred})

In [60]:
res_df[(res_df.rating - res_df.rating_pred)>=2].shape

(8, 3)

In [61]:
res_df[(res_df.rating - res_df.rating_pred)>=2]

,review,rating,rating_pred
7277,life saviours serving excellent food worst tim...,5.0,2.007670
1818,value money ordered second time,5.0,2.970624
4771,not good,5.0,1.998553
16510,may not polished serving packaging etc never b...,5.0,1.894671
14845,oh memories place first drink bangalore almost...,5.0,2.628792
15201,sauce not included,4.0,1.878766
3165,rice quantity less,5.0,2.844196
16515,may not polished serving packaging etc never b...,5.0,1.894671
